In [1]:
from neo4j import GraphDatabase
import os

In [2]:
AURA_CONNECTION_URI = os.getenv('NEO4J_URI')
AURA_USERNAME = os.getenv('NEO4J_USER')
AURA_PASSWORD = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD)
)

In [4]:
import json

with open('mock_data/food.json', 'r') as f:
    foods = json.load(f)

In [12]:
bag_of_words = {}
simple_ingredient = {}

for food, temp in foods.items():
    ingredients = temp['Ingredient']
    materials = [temp.split('(')[0].strip().lower() if '(' in temp else temp.strip().lower() for temp in ingredients]

    if food == "pad_thai": continue
    food = food.replace('_', ' ').lower()
    simple_ingredient[food] = materials
    
    for material in materials:
        if material in bag_of_words:
            bag_of_words[material] += 1
        else:
            bag_of_words[material] = 1

In [13]:
bag_of_words

{'mackerel': 1,
 'chili': 1,
 'garlic': 4,
 'shallots': 2,
 'shrimp paste': 1,
 'lime juice': 4,
 'vegetables': 1,
 'rice flour': 1,
 'coconut milk': 5,
 'sugar': 6,
 'salt': 2,
 'spring onions': 2,
 'chicken': 3,
 'turmeric powder': 1,
 'curry powder': 1,
 'peanut sauce': 1,
 'cucumber relish': 1,
 'fish paste': 1,
 'red curry paste': 1,
 'kaffir lime leaves': 3,
 'green beans': 1,
 'egg': 3,
 'fish sauce': 6,
 'grilled pork': 1,
 'chili powder': 1,
 'roasted rice powder': 1,
 'mint leaves': 1,
 'cilantro': 2,
 'wide rice noodles': 1,
 'soy sauce': 2,
 'chinese broccoli': 1,
 'pork or chicken': 1,
 'glutinous rice flour': 1,
 'palm sugar': 3,
 'grated coconut': 1,
 'water': 3,
 'soft tofu': 1,
 'ginger': 1,
 'massaman curry paste': 1,
 'beef': 1,
 'potatoes': 1,
 'onions': 1,
 'peanuts': 2,
 'cashew nuts': 1,
 'cinnamon': 1,
 'star anise': 1,
 'tamarind paste': 1,
 'shrimp': 1,
 'lemongrass': 1,
 'galangal': 1,
 "thai bird's eye chili": 2,
 'mushroom': 1,
 'green papaya': 1,
 'tamarin

In [14]:
similar_words = {}

for words in bag_of_words.keys():
    list_words = []
    for temp in bag_of_words.keys():
        if temp.lower() != words.lower() and words.lower() in temp.lower():
            list_words.append(temp.lower())
    
    if len(list_words) > 0:
        similar_words[words.lower()] = list_words

similar_words

{'chili': ['chili powder', "thai bird's eye chili"],
 'rice flour': ['glutinous rice flour'],
 'sugar': ['palm sugar'],
 'chicken': ['pork or chicken', 'chicken or shrimp'],
 'egg': ['thai eggplant'],
 'onions': ['spring onions'],
 'shrimp': ['shrimp paste', 'chicken or shrimp'],
 'tamarind': ['tamarind paste'],
 'onion': ['spring onions', 'onions'],
 'lime': ['lime juice', 'kaffir lime leaves'],
 'cucumber': ['cucumber relish']}

In [15]:
def add_food_with_ingredients(tx, food_name, ingredients):
    query = """
    CREATE (a:Food {name: $food_name})
    WITH a
    UNWIND $ingredients AS ingredient
    MERGE (b:Ingredient {name: ingredient}) 
    MERGE (b)-[:RECIPE]->(a)
    """
    tx.run(query, food_name=food_name, ingredients=ingredients)

# Iterating through food and ingredients
for food, ingredients in simple_ingredient.items():
    with driver.session() as session:
        session.write_transaction(add_food_with_ingredients, food, ingredients)

/tmp/ipykernel_15489/3250473487.py:14: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_food_with_ingredients, food, ingredients)


In [16]:
def add_ingredient_relations(tx, ingredient_term, ingredient_variants):
    query = """
    MATCH (a:Ingredient {name: $ingredient_term})
    UNWIND $ingredient_variants AS variant
    MERGE (b:Ingredient {name: variant})
    MERGE (a)-[:SYNONYM]->(b)  // Consider changing to a more appropriate relationship type if necessary
    """ 
    tx.run(query, ingredient_term=ingredient_term, ingredient_variants=ingredient_variants)

for ingredient_term, ingredient_variants in similar_words.items():
    with driver.session() as session:
        session.write_transaction(add_ingredient_relations, ingredient_term, ingredient_variants)

/tmp/ipykernel_15489/790334638.py:12: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(add_ingredient_relations, ingredient_term, ingredient_variants)


In [ ]:
driver.close()

Mock user data for recommendation system

In [69]:
import json

N = 500

users = [f"user_{i}" for i in range(N)]
with open('mock_data/food.json', 'r') as f: data = json.load(f)
foods = list(data.keys())

user_to_index = {user:i for i, user in enumerate(users)}
index_to_user = {i:user for i, user in enumerate(users)}

food_to_index = {user:i for i, user in enumerate(foods)}
index_to_food = {i:user for i, user in enumerate(foods)}

In [70]:
import numpy as np

Matrix = np.zeros((N, len(foods))).astype(int)
Matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(500, 15))

In [71]:
index_not_halal = []
for food in foods:
    ingredients = data[food]["Ingredient"]
    for ingredient in ingredients:
        if "pork" in ingredient.lower():
            print(food)
            index_not_halal.append(food_to_index[food])
            break

print(index_not_halal)

nam_tok_moo
pad_see_ew
[5, 6]


In [72]:
import random

number_islam = random.randint(1, 150)
print(f"Number of Islam: {number_islam}")

islam_users = []
for _ in range(number_islam):
    while True:
        index = random.randint(0, N - 1)
        user = f"user_{index}"
        if user not in islam_users:
            islam_users.append(user)
            break

print(islam_users)

Number of Islam: 95
['user_7', 'user_266', 'user_367', 'user_359', 'user_392', 'user_274', 'user_93', 'user_42', 'user_95', 'user_452', 'user_241', 'user_492', 'user_120', 'user_251', 'user_8', 'user_198', 'user_254', 'user_304', 'user_178', 'user_396', 'user_155', 'user_451', 'user_141', 'user_447', 'user_320', 'user_62', 'user_41', 'user_187', 'user_482', 'user_6', 'user_206', 'user_3', 'user_489', 'user_184', 'user_34', 'user_466', 'user_448', 'user_199', 'user_55', 'user_368', 'user_470', 'user_160', 'user_419', 'user_342', 'user_67', 'user_149', 'user_436', 'user_350', 'user_174', 'user_262', 'user_301', 'user_303', 'user_356', 'user_424', 'user_355', 'user_12', 'user_256', 'user_399', 'user_428', 'user_372', 'user_473', 'user_36', 'user_5', 'user_455', 'user_91', 'user_106', 'user_491', 'user_398', 'user_129', 'user_314', 'user_203', 'user_278', 'user_463', 'user_330', 'user_37', 'user_208', 'user_479', 'user_28', 'user_253', 'user_336', 'user_172', 'user_113', 'user_300', 'user_

In [73]:
for user in islam_users:
    index = user_to_index[user]
    if user in islam_users:
        for food in foods:
            if food_to_index[food] in index_not_halal:
                Matrix[index][food_to_index[food]] -= 1

print(Matrix[7])

[ 0  0  0  0  0 -1 -1  0  0  0  0  0  0  0  0]


In [74]:
for user in users:
    i = user_to_index[user]
    food_list = Matrix[i]
    for index in range(len(food_list)):
        found = random.choice([True, False])
        if found:
            frequency = food_list[index]
            random_freq = random.randint(1, 10)
            if frequency == 0: food_list[index] = random_freq

print(Matrix[7])

[ 0  7  3  0  0 -1 -1  0  0  9  0  7  0  5  5]


In [75]:
Matrix

array([[ 0,  1,  2, ...,  0,  0,  5],
       [ 0,  9,  3, ...,  6,  8,  9],
       [ 0,  4,  0, ...,  0, 10,  0],
       ...,
       [ 0,  6,  0, ...,  0,  0,  3],
       [ 2,  0,  3, ...,  0,  5,  0],
       [10,  0,  0, ...,  1,  1,  0]], shape=(500, 15))

In [91]:
import numpy as np
import json

class FoodManager:
    def __init__(self, users, foods, islam_users, matrix):
        self.users = users
        self.foods = foods
        self.islam_users = set(islam_users)
        self.matrix = np.array(matrix, dtype=int)

    def add_or_update_user(self, user, food_preferences):
        if user not in self.users:
            self.users.append(user)
            new_row = np.zeros(len(self.foods), dtype=int)
            self.matrix = np.vstack([self.matrix, new_row])

        user_index = self.users.index(user)
        for food, preference in food_preferences.items():
            if food in self.foods:
                food_index = self.foods.index(food)
                self.matrix[user_index, food_index] = preference

    def save_to_json(self, filename):
        data = {
            "users": self.users,
            "foods": self.foods,
            "islam_users": list(self.islam_users),
            "matrix": [[int(value) for value in row] for row in self.matrix]
        }
        with open(filename, "w") as file:
            json.dump(data, file)

    @classmethod
    def load_from_json(cls, filename):
        with open(filename, "r") as file:
            data = json.load(file)
        return cls(data["users"], data["foods"], data["islam_users"], data["matrix"])

    def get_user_preferences(self, user):
        print(user)
        if user not in self.users:
            return None
        user_index = self.users.index(user)
        return {self.foods[i]: self.matrix[user_index, i] for i in range(len(self.foods))}

    def is_islam_compliant(self, user):
        return user in self.islam_users if user in self.users else None


In [97]:
manager = FoodManager(users= users, foods= foods, islam_users= islam_users, matrix= Matrix)
manager.save_to_json("mock_data/matrix.json")

In [98]:
manager.get_user_preferences("user_0")

user_0


{'nam_prik_pla_too': np.int64(0),
 'khanom_krok': np.int64(1),
 'pad_thai': np.int64(2),
 'satay_gai': np.int64(5),
 'tod_mun_pla': np.int64(0),
 'nam_tok_moo': np.int64(0),
 'pad_see_ew': np.int64(4),
 'khanom_tom': np.int64(0),
 'tao_huay': np.int64(2),
 'massaman': np.int64(4),
 'tom_yum_goong': np.int64(0),
 'som_tum': np.int64(0),
 'khao_pad': np.int64(0),
 'gaeng_keow_wan_gai': np.int64(0),
 'bai_toey': np.int64(5)}